# ReadyCT Workshop: Hands-On AI

Welcome to this hands-on coding workshop! In this notebook, we’ll explore how artificial intelligence (AI) can recognize drawings — just like how people do.

We'll be using a real-world dataset from Google called the [Quick, Draw! dataset](https://quickdraw.withgoogle.com/data), which contains millions of doodles collected from people around the world. We'll focus on distinguishing between two types of doodles: cats and dogs.

By the end of this notebook, you’ll have trained your own machine learning models to classify images — including a simple neural network!

## What Is AI and How Does It Recognize Images?
AI refers to computer programs that can learn from data and make decisions or predictions. One common use of AI is image recognition — for example, teaching a computer to tell the difference between a picture of a cat and a dog.

But how can a machine "see" a drawing?

An image is really just a grid of numbers (pixels).

AI learns patterns from these images.

Once trained, it can make predictions on new, unseen images.

Today, we’ll feed our models simple black-and-white doodles, and they’ll learn to guess: is this a cat or a dog?

First, we start by importing all necessary packages. If you get an error, do `pip install PACKAGE_NAME`, where `PACKAGE_NAME` is the name of the missing package. For `sklearn`, you have to do `pip install scikit-learn`.

In [ ]:
import os
import random
import urllib.request

import numpy as np
import ndjson
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

import utils

## Loading the Doodle Dataset

We'll start by downloading images of cats and dogs from the Doodle dataset. We have pre-written some code that automatically downloads the data, and prepares it to feed into our machine learning models. If you are interested in 'peeking under the hood', check out the `utils.py` program to see how we downloaded and prepared our data!

As a general guide, these are the steps that we do to prepare our dataset:

1. Combine the cat and dog images into one single dataset (instead of two different datasets).
2. Add a label for each image:
   - `0` for **cat**
   - `1` for **dog**
3. Normalize the images by dividing all pixel values by 255.
   - This scales the data to be between `0` and `1`, which helps models learn better.
4. Split the data into training and test sets:
   - The training set is used to teach the model.
   - The test set is used to evaluate how well the model works on new data.
   - We split the dataset into 80% training and 20% testing.

In [ ]:
# downloading and visualizing the data - this might take some time!
# preparing the data to feed into the models
X, y = utils.download_and_preprocess_data(doodles=True)

# splitting into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Model #1: Logistic Regression

Logistic Regression is a simple yet powerful machine learning model used for classification tasks. It tries to find a decision boundary that separates two classes — in our case, cats and dogs — based on pixel values.

This will serve as our baseline model. Later, we’ll compare its performance with more advanced methods.

Here's a simple picture to think about how Logistic Regression does its job recognizing cats as cats and dogs as dogs.

![logistic regression](figs/logreg.png)

In [ ]:
# creating our Logistic Regression model
clf = LogisticRegression(max_iter=1000)

# feeding our training data to the model
clf.fit(X_train, y_train)

# feeding our test data to the model and getting the accuracy of the model
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc * 100:.2f}%")
utils.show_10_random_predictions(X_test, y_test, clf, model_type='sklearn', model_name='Logistic Regression', accuracy=acc * 100, dataset_type='doodles')

So we see that our Logistic Regression model achieves an accuracy between 60-70%. This means that the model can correctly predict whether the image is a doodle of a cat or a dog 60-70% of the time (exact accuracy is what your model outputs). We will now compare these results against two other models.

## Model #2: Support Vector Machine (SVM)

A Support Vector Machine (SVM) is another type of classification model. It works by finding the best boundary that separates two classes of data.

Unlike logistic regression, which models probabilities, an SVM tries to maximize the separation between the two classes. This can make it more effective when the classes are not easily separable with a simple line or curve.

We'll train the SVM on the same data and compare its performance to the logistic regression model.

Again, here's a simple picture to get you thinking about what SVM is going. It also tries to find a boundary between cat and dog data points like Logistic Regression, but also tries to maximize the separation between the boundary, to really confidently say "yes, this is a cat", or "this is really definitely a dog".

![SVM](figs/svm.png)

In [ ]:
# creating our SVM model
svm_clf = SVC(kernel='linear')  

# feeding our training data to the model
svm_clf.fit(X_train, y_train)

# feeding our test data to the model and getting the accuracy of the model
y_pred_svm = svm_clf.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {svm_acc * 100:.2f}%")
utils.show_10_random_predictions(X_test, y_test, svm_clf, model_type='sklearn', model_name='SVM', accuracy=svm_acc * 100, dataset_type='doodles')

Now we see that our SVM model also achieves an accuracy between 60-70%. This means that the model can correctly predict whether the image is a doodle of a cat or a dog 60-70% of the time (exact accuracy is what your model outputs). We will now compare these results against one more model.

## Model #3: Neural Network (Simple ResNet)

Neural networks are *powerful* models that learn by stacking layers of computations. A ResNet, short for Residual Network, is a type of deep neural network that is widely used by ML practitioners as a standard benchmark model. See [this](https://en.wikipedia.org/wiki/Residual_neural_network) if you're interested in learning more about ResNet!

In this notebook, we use a very simple version of a ResNet. It includes convolutional layers that detect patterns in the images, and a residual block that helps the model learn more complex features without losing important information.

We'll train this network using [PyTorch](https://pytorch.org/), a deep learning library widely used in both research and industry. Then, we'll compare its accuracy to the other models.

We start by preparing the data specially for this model, as it requires a different format than what we've used for Logistic Regression and SVM. Again, we have pre-written a program in `utils.py` that takes care of the data pre-processing for you.

In [ ]:
# preparing the data to feed to the ResNet
train_loader, test_loader = utils.preprocess_pytorch_data(X, y)

Now we have to define the model architecture. This is like writing a recipe for a cake, where the cake is our neural network, and we write down what ingredinents go into baking this cake and in what order. We have drawn a simple version of what this model architecture looks like, but don't worry about the details - think of the ResNet as a food processor which takes in your image (which is either a cat or a dog), does some stuff (learns features from the image), and then finally outputs a prediction: either 0 if the model thinks the image is a cat, or 1 if the model think the image is a dog.

![simple ResNet](figs/simple_resnet.png)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)
    
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity  # Residual connection
        out = self.relu(out)
        return out

class SimpleResNet(nn.Module):
    def __init__(self, in_channels=1):
        """
        Args:
            in_channels (int): Number of input channels (1 for doodles, 3 for CIFAR)
        """
        super().__init__()
        self.conv = nn.Conv2d(in_channels, 16, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.resblock = ResidualBlock(16)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(16, 2)  # 2 classes (cat/dog)
    
    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))
        x = self.resblock(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

Now with the model architecture defined, we will start setting the scene for training our model.

In [ ]:
# set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# creating our ResNet model from the architecture defined above
model = SimpleResNet(in_channels=1).to(device)

# defining a "loss function" that helps the model learn patterns from the images
criterion = nn.CrossEntropyLoss()

# defining an "optimizer" that also helps the model learn patterns
optimizer = optim.Adam(model.parameters(), lr=0.001)

Now with everything set in place, we train the ResNet model in a 'training loop'. We feed the data through the neural network 5 times (also called 'epochs') to help it learn complex patterns from the data.

In [ ]:
train_accuracies = []

# Training loop
epochs = 5  # adjust as needed
model.train()

for epoch in range(epochs):
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_acc = correct / total
    train_accuracies.append(epoch_acc)

    print(f"Epoch {epoch+1}/{epochs} - Accuracy: {epoch_acc * 100:.2f}%")

# Plotting epoch vs accuracy
utils.plot_accuracy_curve(train_accuracies, title="Epoch vs Training Accuracy")

Now that we have trained the model, we will feed it the test data, and see how well it can perform.

In [ ]:
# Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# calculating accuracy and displaying model's predictions
accuracy = 100 * correct / total
print(f"Simple ResNet Test Accuracy: {accuracy:.2f}%")
utils.show_10_random_predictions(X_test, y_test, model, model_type='resnet', model_name='ResNet', accuracy=accuracy, dataset_type='doodles')  # model is your PyTorch model

## Why Did the ResNet Perform Worse?

Even though neural networks like ResNet are more powerful and flexible, they don't always perform better — especially on small or simple datasets like ours.

There are a few reasons why the ResNet may have lower accuracy than the logistic regression and SVM models:

- The dataset is small (only 1,000 images total), which makes it harder for deep models to learn effectively.
- Neural networks have a lot more parameters and need more data to generalize well.
- Our ResNet was trained for just a few epochs and kept very simple for this workshop. A more complex version with more layers and training time might perform better.
- Simpler models like logistic regression and SVM often work surprisingly well when the input features (pixel patterns) are already quite informative.

This is an important lesson in AI: sometimes, **simpler models are better suited to the problem** — especially when data is limited. But for large, complex datasets (like natural photos or videos), deep neural networks usually shine.

Note here that the ResNet model achieves around ~50% accuracy. This means it's randomly guessing what the image is - either it will be a cat or a dog. This is like when you flip a coin and you can guess correctly if it's heads or tails 50% of the time.

Also note that we only trained our model here for 5 epochs. What happens if we increase that number? We ran that experiment for you, here are the results when we train the model for 10,000 epochs on a powerful GPU.

We find that the accuracy starts low, around 55%, but then slowly starts increasing to or near 100%, stabilizing around the 200th epoch. This means the model has learned meaningul features by the 200th epoch, enough to be correctly training the model.

![trained for 10000 epochs](figs/01_doodles_acc.png)

But what happens when we test our model on the test images? We find that it achieves a higher accuracy than our 5 epoch-trained model, 72%. This shows us that more time training the model on the same small dataset *does* help in increasing the accuracy a little bit.

![tested on 50 images](figs/02_doodles_test.png)

## Conclusion

In this notebook, we explored how artificial intelligence can be used to recognize hand-drawn doodles. We trained and tested three different models: logistic regression, support vector machine (SVM), and a simple neural network based on the ResNet architecture.

Along the way, we learned how to prepare image data, train models, and evaluate their performance. We also saw how even simple models can perform well on visual tasks, and how more advanced models like neural networks can capture more complex patterns.

This hands-on activity is just the beginning. The same ideas used here are part of how modern AI systems recognize faces, understand handwriting, power self-driving cars, and more. With practice and curiosity, you can keep exploring and building even more powerful AI models.


================================================================================
================================================================================

## Bonus Content! Classifying Real Images

We just classified *doodles* of cats and dogs using three models (Logistic Regression, SVM, ResNet). But what if we now use *real images* of cats and dogs? Isn't that exciting?

We'll be using the [CIFAR-10 dataset](https://en.wikipedia.org/wiki/CIFAR-10), which is a famous benchmark dataset of images used in machine learning. There's 10 different classes of images: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. But we're going to use a small subset of cats and dogs only. 

We'll do the same thing as before, which is pre-processing the data, and training Logistic Regression, SVM, and ResNet models on it.

In [ ]:
# downloading and visualizing the data - this might take some time!
# preparing the data to feed into the models
X, X_flat, y = utils.download_and_preprocess_data(doodles=False)

# splitting into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2)

WHOA!?!? Why do these images look so...blurry? 

Our laptops often aren't powerful enough (as in, they don't have enough memory and GPU compute power) to handle processing very large datasets. So to keep a light load on your laptops, we are doing 2 things:
1. using a smaller subset of cats and dogs data - 250 images per class.
2. using a lower resolution.

These images are originally 32x32 pixels (and in color!), however for this workshop, we have lowered the resolution to 28x28 pixels. This is what makes these pictures look blurry. Don't worry - let's see what happens! Will our models be able to recognize these cats as cats and dogs as dogs? Only one way to find out!

## Model #1: Logistic Regression

Let's run these images through a new Logistic Regression model.

In [ ]:
# creating our Logistic Regression model
clf = LogisticRegression(max_iter=1000)

# feeding our training data to the model
clf.fit(X_train, y_train)

# feeding our test data to the model and getting the accuracy of the model
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc * 100:.2f}%")
utils.show_10_random_predictions(X_test, y_test, clf, model_type='sklearn', model_name='Logistic Regression', accuracy=acc * 100, dataset_type='cifar')

## Model #2: Support Vector Machine (SVM)

Again, we'll repeat the experiment for SVM...

In [ ]:
# creating our SVM model
svm_clf = SVC(kernel='linear')  

# feeding our training data to the model
svm_clf.fit(X_train, y_train)

# feeding our test data to the model and getting the accuracy of the model
y_pred_svm = svm_clf.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {svm_acc * 100:.2f}%")
utils.show_10_random_predictions(X_test, y_test, svm_clf, model_type='sklearn', model_name='SVM', accuracy=svm_acc * 100, dataset_type='cifar')

## Model #3: Neural Network (Simple ResNet)

Finally, we'll train our deep neural network, Simple ResNet. Note that we have to pre-process the data to get it into a format the neural network can work with, but we've taken care of that for you, so you just need to run the code!

In [ ]:
# splitting into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
train_loader, test_loader = utils.preprocess_pytorch_data(X, y)

We don't have to define the architecture of the ResNet again, since it has already been defined earlier and we're not making any changes to the model architecture. We will need to re-instantiate the model, loss function, and optimizer, however.

In [ ]:
# set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# creating our ResNet model from the architecture defined above
model = SimpleResNet(in_channels=3).to(device)

# defining a "loss function" that helps the model learn patterns from the images
criterion = nn.CrossEntropyLoss()

# defining an "optimizer" that also helps the model learn patterns
optimizer = optim.Adam(model.parameters(), lr=0.001)

Once again, we now start to train the model with the cats and dogs images.

In [ ]:
train_accuracies = []

# Training loop
epochs = 5  # adjust as needed
model.train()

for epoch in range(epochs):
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_acc = correct / total
    train_accuracies.append(epoch_acc)

    print(f"Epoch {epoch+1}/{epochs} - Accuracy: {epoch_acc * 100:.2f}%")

# Plotting epoch vs accuracy
utils.plot_accuracy_curve(train_accuracies, title="Epoch vs Training Accuracy")

And final touches...*test* the model to see how well it learned to recognize cat and dog pictures!

In [ ]:
# Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# calculating accuracy and displaying model's predictions
accuracy = 100 * correct / total
print(f"Simple ResNet Test Accuracy: {accuracy:.2f}%")
utils.show_10_random_predictions(X_test, y_test, model, model_type='resnet', model_name='ResNet', accuracy=accuracy, dataset_type='cifar')  # model is your PyTorch model

![trained for 10000 epochs](figs/03_cifar_acc.png)
![tested on 50 images](figs/04_cifar_test.png)

BUMP UP RES TO 32
\


kmeans, decision tree, random forest, etc ==> EXPLAIANBLE!!!!!!
vits?

add resources for them to learn more eg colab, skit learn docs, wikipedia, how to use with python
enable them to learn things by theseleves 


**Exercise for you:** check the accuracies for each of the models you just ran (Logistic Regression, SVM, ResNet) on this CIFAR-10 cats and dogs dataset, with the same models you ran for the Doodles dataset. 

- Which model, out of the 6, performed the best?
- Did the models trained on the different datasets perform comparably?
- What do you think of the difference in performances between the models and datasets? 

Remember the difference between both datasets - the Doodles dataset is a very simple black-and-white dataset of doodles, wheareas the CIFAR-10 dataset is a color dataset of pictures of real cats and dogs, which contains more information.

# Resources to learn more